In [6]:
import torch.nn.functional as F
from catalyst.contrib import registry
from torch import nn
import torch

def log_t(u, t):
    """Compute log_t for `u`."""

    if t == 1.0:
        return torch.log(u)
    else:
        return (u ** (1.0 - t) - 1.0) / (1.0 - t)


def exp_t(u, t):
    """Compute exp_t for `u`."""

    if t == 1.0:
        return torch.exp(u)
    else:
        return torch.relu(1.0 + (1.0 - t) * u) ** (1.0 / (1.0 - t))


def compute_normalization_fixed_point(activations, t, num_iters=5):
    """Returns the normalization value for each example (t > 1.0).
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (> 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    mu = torch.max(activations, dim=-1).values.view(-1, 1)
    normalized_activations_step_0 = activations - mu

    normalized_activations = normalized_activations_step_0
    i = 0
    while i < num_iters:
        i += 1
        logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)
        normalized_activations = normalized_activations_step_0 * (logt_partition ** (1.0 - t))

    logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)

    return -log_t(1.0 / logt_partition, t) + mu


def compute_normalization(activations, t, num_iters=5):
    """Returns the normalization value for each example.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (< 1.0 for finite support, > 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    if t < 1.0:
        return None # not implemented as these values do not occur in the authors experiments...
    else:
        return compute_normalization_fixed_point(activations, t, num_iters)


def tempered_softmax(activations, t, num_iters=5):
    """Tempered softmax function.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature tensor > 0.0.
    num_iters: Number of iterations to run the method.
    Returns:
    A probabilities tensor.
    """

    if t == 1.0:
        normalization_constants = torch.log(torch.sum(torch.exp(activations), dim=-1))
    else:
        normalization_constants = compute_normalization(activations, t, num_iters)

    return exp_t(activations - normalization_constants, t)


def bi_tempered_logistic_loss(activations, labels, t1, t2, label_smoothing=0.0, num_iters=5):

    """Bi-Tempered Logistic Loss with custom gradient.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    labels: A tensor with shape and dtype as activations.
    t1: Temperature 1 (< 1.0 for boundedness).
    t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
    label_smoothing: Label smoothing parameter between [0, 1).
    num_iters: Number of iterations to run the method.
    Returns:
    A loss tensor.
    """

    if label_smoothing > 0.0:
        num_classes = labels.shape[-1]
        labels = (1 - num_classes / (num_classes - 1) * label_smoothing) * labels + label_smoothing / (num_classes - 1)

    probabilities = tempered_softmax(activations, t2, num_iters)

    temp1 = (log_t(labels + 1e-10, t1) - log_t(probabilities, t1)) * labels
    temp2 = (1 / (2 - t1)) * (torch.pow(labels, 2 - t1) - torch.pow(probabilities, 2 - t1))
    loss_values = temp1 - temp2

    return torch.sum(loss_values, dim=-1)
   
    
@registry.Criterion
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smooth_factor=0.05):
        super().__init__()
        self.smooth_factor = smooth_factor

    def _smooth_labels(self, num_classes, target):
        # When label smoothing is turned on,
        # KL-divergence between q_{smoothed ground truth prob.}(w)
        # and p_{prob. computed by model}(w) is minimized.
        # If label smoothing value is set to zero, the loss
        # is equivalent to NLLLoss or CrossEntropyLoss.
        # All non-true labels are uniformly set to low-confidence.

        target_one_hot = F.one_hot(target, num_classes).float()
        target_one_hot[target_one_hot == 1] = 1 - self.smooth_factor
        target_one_hot[target_one_hot == 0] = self.smooth_factor
        return target_one_hot

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        target_one_hot = self._smooth_labels(input.size(1), target)
        return F.kl_div(logp, target_one_hot, reduction='sum')

@registry.Criterion
class TemperedLogLoss(nn.Module):
    def __init__(self, label_smoothing=0.05, t1 = 0.7, t2 = 2, num_iters = 5):
        super().__init__()
        self.label_smoothing = label_smoothing
        self.t1 = t1
        self.t2 = t2
        self.num_iters = num_iters

    def forward(self, input, target):
        return bi_tempered_logistic_loss(input, target, self.t1, self.t2, self.label_smoothing, self.num_iters)

RegistryException: Factory with name 'LabelSmoothingLoss' is already present
Already registered: '<class '__main__.LabelSmoothingLoss'>'
New: '<class '__main__.LabelSmoothingLoss'>'

In [1]:
import albumentations as A
from albumentations.pytorch import ToTensor
from torch.utils.data import Dataset, DataLoader
import cv2

def get_super_light_augmentations(image_size):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ])

def get_light_augmentations(image_size):
    min_size = min(image_size[0], image_size[1])
    return A.Compose([
        A.RandomSizedCrop(min_max_height=(int(min_size * 0.85), min_size),
                          height=image_size[0],
                          width=image_size[1], p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ])

def get_medium_augmentations(image_size):
    min_size = min(image_size[0], image_size[1])

    return A.Compose([
        A.OneOf([A.RandomSizedCrop(min_max_height=(int(min_size* 0.85), min_size),
                          height=image_size[0],
                          width=image_size[1]),
                A.Resize(image_size[0], image_size[1]),
                A.CenterCrop(image_size[0], image_size[1])
                ], p = 1.0),

        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.15,
                                       contrast_limit=0.5),
            A.RandomGamma(gamma_limit=(50, 150)),
            A.NoOp()
        ], p = 1.0),
        
        A.OneOf([A.CLAHE(p=0.5, clip_limit=(10, 10), tile_grid_size=(3, 3)),
                A.FancyPCA(alpha=0.4),
                A.NoOp(),
                ], p = 1.0),
        
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Flip(p = 0.5)
    ])

def get_hard_augmentations(image_size):
    return None

# def get_medium_augmentations(image_size):
#     return A.Compose([
#         A.OneOf([
#             A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
#                                rotate_limit=15,
#                                border_mode=cv2.BORDER_CONSTANT, value=0),
#             A.OpticalDistortion(distort_limit=0.11, shift_limit=0.15,
#                                 border_mode=cv2.BORDER_CONSTANT,
#                                 value=0),
#             A.NoOp()
#         ]),
#         A.RandomSizedCrop(min_max_height=(int(image_size[0] * 0.75), image_size[0]),
#                           height=image_size[0],
#                           width=image_size[1], p=0.3),
#         A.OneOf([
#             A.RandomBrightnessContrast(brightness_limit=0.5,
#                                        contrast_limit=0.4),
#             A.RandomGamma(gamma_limit=(50, 150)),
#             A.NoOp()
#         ]),
#         A.OneOf([
#             A.RGBShift(r_shift_limit=20, b_shift_limit=15, g_shift_limit=15),
#             A.HueSaturationValue(hue_shift_limit=5,
#                                  sat_shift_limit=5),
#             A.NoOp()
#         ]),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5)
#     ])

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
#     return [ToTensor()]
    return A.Compose([A.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225)), ToTensor()])

# def compose(transforms_to_compose):
#     # combine all augmentations into one single pipeline
#     result = A.Compose([
#       item for sublist in transforms_to_compose for item in sublist
#     ])
#     return result


def get_test_transform(image_size):
    return A.Compose([A.Resize(image_size[0], image_size[1], p = 1.0), A.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p = 1.0), ToTensor()])

def get_train_transform(augmentation, image_size):
    LEVELS = {
        'super_light': get_super_light_augmentations,
        'light': get_light_augmentations,
        'medium': get_medium_augmentations,
        'hard': get_hard_augmentations,
#         'hard2': get_hard_augmentations_v2
    }

    aug = LEVELS[augmentation](image_size)

    return A.Compose([aug, post_transforms()])
    

In [8]:
import math
import os
from typing import Tuple, List

import albumentations as A
import cv2
import numpy as np
import pandas as pd
# from pytorch_toolbelt.utils import fs
# from pytorch_toolbelt.utils.fs import id_from_fname
# from pytorch_toolbelt.utils.torch_utils import tensor_from_rgb_image
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils import compute_sample_weight
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler



def prepare_by_stretch_img(img, img_size):
    shape = (img.shape[1], img.shape[2])

    if shape[0] < img_size[0] or shape[1] < img_size[1]:
        coef_0 = float(img_size[0]) / shape[0]
        coef_1 = float(img_size[1]) / shape[1]

        coef = int(max(coef_0, coef_1)) + 1

        return cv2.resize(img, (shape[0]*coef, shape[1]*coef))
        
    return img

def prepare_by_reshape(img, img_size):
    return cv2.resize(img, (img_size[0], img_size[1]))

UNLABELED_CLASS = -100

def get_prepare_function(prepare_function_name):
    d = {
        'reshape': prepare_by_reshape,
        'stretch': prepare_by_stretch_img,
    }
    return d[prepare_function_name]

class TaskDataset(Dataset):
    def __init__(self, images, targets, img_size, prepare_function_name,
                 transform: A.Compose,
                 target_as_array=False,
                 dtype=int):
        if targets is not None:
            targets = np.array(targets)
            unique_targets = set(targets)
            if len(unique_targets.difference({0, 1, 2, 3, 4, UNLABELED_CLASS})):
                raise ValueError('Unexpected targets in Y ' + str(unique_targets))

        self.images = np.array(images)
        self.targets = targets
        self.transform = transform
        self.target_as_array = target_as_array
        self.dtype = dtype
        self.img_size = img_size
        self.prepare_function_name = prepare_function_name

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        image = cv2.imread(self.images[item])  # Read with OpenCV instead PIL. It's faster
        if image is None:
            raise FileNotFoundError(self.images[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        prep = get_prepare_function(self.prepare_function_name)
        image = prep(image, self.img_size)
        height, width = image.shape[:2]
        label = UNLABELED_CLASS

        if self.targets is not None:
            label = self.targets[item]

        #Плохо тут это делать но пока так
        # image = prepare_img(image, self.img_size)
        data = self.transform(image=image, label=label)
        label = data['label']

        data = {'image': data['image']}

        label = self.dtype(label)
        if self.target_as_array:
            data['targets'] = np.array([label])
        else:
            data['targets'] = label

        return data





def split_train_valid(x, y, fold=None, folds=4, random_state=42):
    """
    Common train/test split function
    :param x:
    :param y:
    :param fold:
    :param folds:
    :param random_state:
    :return:
    """
    train_x, train_y = [], []
    valid_x, valid_y = [], []

    if fold is not None:
        assert 0 <= fold < folds
        skf = StratifiedKFold(n_splits=folds, random_state=random_state, shuffle=True)

        for fold_index, (train_index, test_index) in enumerate(skf.split(x, y)):
            if fold_index == fold:
                train_x = x[train_index]
                train_y = y[train_index]
                valid_x = x[test_index]
                valid_y = y[test_index]
                break
    else:
        train_x, valid_x, train_y, valid_y = train_test_split(x, y,
                                                              random_state=random_state,
                                                              test_size=1.0 / folds,
                                                              shuffle=True,
                                                              stratify=y)

    assert len(train_x) and len(train_y) and len(valid_x) and len(valid_y)
    assert len(train_x) == len(train_y)
    assert len(valid_x) == len(valid_y)
    return train_x, valid_x, train_y, valid_y


def get_current_train(data_dir):
    df = pd.read_csv(os.path.join(data_dir, 'train_images.csv'))
    x = np.array(df['image_id'].apply(lambda x: os.path.join(data_dir, 'train_images', f'{x}')))
    y = np.array(df['label'], dtype=int)
    return x, y


def get_extra_train(data_dir):
    df = pd.read_csv(os.path.join(data_dir, 'extra_images.csv'))
    x = np.array(df['id_code'].apply(lambda x: os.path.join(data_dir, 'extra_data', f'{x}')))
    y = np.array(df['label'], dtype=int)
    return x, y


def get_unlabeled(dataset_dir, healthy_eye_fraction=1):
    df= pd.read_csv(os.path.join(dataset_dir, 'unlabeled.csv'))
    x = np.array(df['image_id'].apply(lambda x: os.path.join(data_dir, 'unlabeled', f'{x}')))
    y = np.array([UNLABELED_CLASS] * len(x), dtype=int)
    return x, y






def append_train_test(existing, to_add):
    train_x, train_y, valid_x, valid_y = existing
    tx, vx, ty, vy = to_add
    train_x.extend(tx)
    train_y.extend(ty)
    valid_x.extend(vx)
    valid_y.extend(vy)
    return train_x, train_y, valid_x, valid_y


def get_dataset(datasets: List[str], data_dir='data', random_state=42):
    """
    :param datasets: List "aptos2015-train/fold0", "messidor",
    :param fold:
    :param folds:
    :return:
    """

    all_x = []
    all_y = []
    sizes = []

    for ds in datasets:

        dataset_name = ds

        if dataset_name == 'current_train':
            x, y = get_current_train(data_dir)
        elif dataset_name == 'extra_train':
            x, y = get_extra_train(data_dir)
        elif dataset_name == 'unlabeled':
            x, y = get_aptos2015_test(data_dir)
        else:
            raise ValueError(dataset_name)

        all_x.extend(x)
        all_y.extend(y)
        sizes.append(len(x))

    return all_x, all_y, sizes


def get_datasets_universal(
        train_on: List[str],
        valid_on: List[str],
        data_dir='data',
        prep_function = 'reshape',
        image_size=(512, 512),
        augmentation='medium',
        preprocessing=None,
        target_dtype=int,
        random_state=42,
        coarse_grading=False,
        folds=4) -> Tuple[TaskDataset, TaskDataset, List]:
    train_x, train_y, sizes = get_dataset(train_on, data_dir=data_dir, random_state=random_state)
    valid_x, valid_y, _ = get_dataset(valid_on, data_dir=data_dir, random_state=random_state)

    train_transform = get_train_transform(augmentation = augmentation, image_size = image_size)
    valid_transform = get_test_transform()

    train_ds = TaskDataset(train_x, train_y, image_size, prep_function,
                                  transform=train_transform,
                                  dtype=target_dtype)

    valid_ds = TaskDataset(valid_x, valid_y, image_size, prep_function,
                                  transform=valid_transform,
                                  dtype=target_dtype)

    return train_ds, valid_ds, sizes


def get_datasets(
        data_dir='data',
        prep_function = 'reshape',
        image_size=(512, 512),
        augmentation='medium',
        use_current=True,
        use_extra=False,
        use_unlabeled=False,
        target_dtype=int,
        random_state=42,
        fold=None,
        folds=4) -> Tuple[TaskDataset, TaskDataset, List]:
    assert use_current or use_extra or use_unlabeled

    trainset_sizes = []
    data_split = [], [], [], []

  

    if use_current:
        x, y = get_current_train(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    if use_extra:
        x, y = get_extra_train(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    if use_unlabeled:
        x, y = get_unlabeled(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    train_x, train_y, valid_x, valid_y = data_split



    train_transform = get_train_transform(augmentation = augmentation, image_size = image_size)
    valid_transform = get_test_transform()

    train_ds = TaskDataset(train_x, train_y, image_size, prep_function,
                                      transform=train_transform,
                                      dtype=target_dtype)

    valid_ds = TaskDataset(valid_x, valid_y, image_size, prep_function,
                                  transform=valid_transform,
                                  dtype=target_dtype)

    return train_ds, valid_ds, trainset_sizes


def get_dataloaders(train_ds, valid_ds,
                    batch_size,
                    num_workers = 1,
                    fast=False,
                    train_sizes=None,
                    balance=False,
                    balance_datasets=False,
                    balance_unlabeled=False,
                    ):
    sampler = None
    weights = None
    num_samples = 0

    if balance_unlabeled:
        labeled_mask = (train_ds.targets != UNLABELED_CLASS).astype(np.uint8)
        weights = compute_sample_weight('balanced', labeled_mask)
        num_samples = int(np.mean(train_sizes))

    if balance:
        weights = compute_sample_weight('balanced', train_ds.targets)
        hist = np.bincount(train_ds.targets)
        min_class_counts = int(min(hist))
        num_classes = len(np.unique(train_ds.targets))
        num_samples = min_class_counts * num_classes

    if balance_datasets:
        assert train_sizes is not None
        dataset_balancing_term = []

        for subset_size in train_sizes:
            full_dataset_size = float(sum(train_sizes))
            dataset_balancing_term.extend([full_dataset_size / subset_size] * subset_size)

        dataset_balancing_term = np.array(dataset_balancing_term)
        if weights is None:
            weights = np.ones(len(train_ds.targets))

        weights = weights * dataset_balancing_term
        num_samples = int(np.mean(train_sizes))

    # If we do balancing, let's go for fixed number of batches (half of dataset)
    if weights is not None:
        sampler = WeightedRandomSampler(weights, num_samples)

    if fast:
        weights = np.ones(len(train_ds))
        sampler = WeightedRandomSampler(weights, 16)

    train_dl = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=sampler is None, sampler=sampler,
                          pin_memory=True, drop_last=True)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False,
                          pin_memory=True, drop_last=False)

    return train_dl, valid_dl

In [9]:
from catalyst.dl import AccuracyCallback
from torch import nn

In [10]:
log_dir = os.path.join('../runs', 'notebook_ex')
os.makedirs(log_dir, exist_ok=False)

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: '../runs\\notebook_ex'

In [11]:
from efficientnet_pytorch import EfficientNet
class EfnB3(nn.Module):
    
    def __init__(self, middle_layer = 1024, output_layer = 5):
        super(EfnB3, self).__init__()
        self.efn = EfficientNet.from_name('efficientnet-b3', include_top = False)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(1536, middle_layer)
        self.fc2 = nn.Linear(middle_layer, output_layer)
    
    def forward(self, x):
        x = self.efn(x)
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        return x
    
model = EfnB3()

In [12]:
model = model.cuda()

In [13]:
train_ds, valid_ds, train_sizes = get_datasets(data_dir='../data',
                                                    use_current=True,
                                                    use_extra=False,
                                                    image_size=(300, 300),
                                                    prep_function = 'reshape',
                                                    augmentation='light',
                                                    target_dtype=int,
                                                    fold=1,
                                                    folds=4)

In [15]:
len(train_ds)

16048

In [9]:
train_loader, valid_loader = get_dataloaders(train_ds, valid_ds,
                                                batch_size=8,
                                                train_sizes=train_sizes,
                                                num_workers = 1,
                                                balance=True,
                                                balance_datasets=True,
                                                balance_unlabeled=False)

In [14]:
next(iter(train_loader))['image'].shape

torch.Size([8, 3, 300, 300])

In [17]:
batch_input = next(iter(train_loader))['image']
batch_target = next(iter(train_loader))['targets']

In [24]:
model = model.cpu()

In [30]:
import torch.nn.functional as F

target = F.one_hot(batch_target, 5).float()
target

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]])

In [19]:
batch_target

tensor([1, 4, 2, 3, 1, 1, 4, 3])

In [25]:
input = model(batch_input)

In [26]:
input

tensor([[ 0.0121,  0.1245,  0.0269, -0.0268, -0.0580],
        [ 0.0302,  0.1763, -0.0499, -0.0464,  0.0049],
        [ 0.2470,  0.3220, -0.4684, -0.0083,  0.2111],
        [ 0.0135,  0.1140,  0.0106, -0.0009, -0.0702],
        [ 0.0328,  0.1236, -0.0239, -0.0121, -0.0089],
        [ 0.0453,  0.1344, -0.1169, -0.0392,  0.0188],
        [ 0.0134,  0.1041,  0.0158, -0.0167, -0.0525],
        [ 0.0168,  0.1253, -0.1604, -0.0255, -0.0202]],
       grad_fn=<AddmmBackward>)

In [35]:
v, l = bi_tempered_logistic_loss(input, target, 0.5, 4, 0.05, 5)

In [40]:
torch.sum(l)

tensor(5.1671, grad_fn=<SumBackward0>)

In [32]:
logp = F.log_softmax(input, dim=1)
target_one_hot = target
F.kl_div(logp, target_one_hot, reduction='sum')

tensor(13.2337, grad_fn=<KlDivBackward>)

In [11]:
import collections
from catalyst.dl import SupervisedRunner, EarlyStoppingCallback
from catalyst.utils import load_checkpoint, unpack_checkpoint

loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = valid_loader

runner = SupervisedRunner(input_key='image')

In [12]:
import catalyst 

# criterions = LabelSmoothingLoss()
criterions = TemperedLogLoss()
# optimizer = catalyst.contrib.nn.optimizers.radam.RAdam(model.parameters(), lr = learning_rate)
optimizer = catalyst.contrib.nn.optimizers.Adam(model.parameters(), lr = 1e-3)
# criterions = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25], gamma=0.8)
# cappa = CappaScoreCallback()
callbacks = [AccuracyCallback(num_classes=5)]


main_metric = 'accuracy01'

runner.train(
    fp16=True,
    model=model,
    criterion=criterions,
    optimizer=optimizer,
    # scheduler=scheduler,
    callbacks=callbacks,
    loaders=loaders,
    logdir=log_dir,
    num_epochs=10,
    verbose=1,
    main_metric=main_metric,
    minimize_metric=False,
)

1/10 * Epoch (train):   0% 0/2006 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (8) must match the size of tensor b (5) at non-singleton dimension 1

# Количество параметров

NameError: name 'model' is not defined